In [ ]:
%load_ext autoreload
import torch
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*weights_only=False.*")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_path = '../Data/unseen/xeroa'
map_path = list(Path(test_path).glob('*.osu'))[0]
P_VERSION = '1.0'
K_VERSION = '1.0'

In [ ]:
# Position
%autoreload 2
from utils.dataparser.PositionData import PositionData
from Model.modelmanager.PositionModel import PositionModel
p_model_config= torch.load(f'./trained_models/[POS]NOSU_{P_VERSION}.pth')
p_data_config = {
    'pos_context_size': p_model_config['hyperparameters']['context_size'],
    'pos_time_window': p_model_config['hyperparameters']['time_window']
}
p_data = PositionData(None, p_data_config)
inputs, p_time, _ = p_data.generate_one(map_path)
p_model = PositionModel(None, None, device, p_model_config)
p_predictions = p_model.predict(inputs)

In [ ]:
# Keypress
%autoreload 2
from utils.dataparser.KeypressData import KeypressData
from Model.modelmanager.KeypressModel import KeypressModel
k_model_config = torch.load(f'./trained_models/[KEY]NOSU_{K_VERSION}.pth')
k_data_config = {
    'key_context_size': k_model_config['hyperparameters']['context_size'],
    'key_time_window': k_model_config['hyperparameters']['time_window']
}
k_data = KeypressData(None, k_data_config)
inputs, k_time, k_end_time = k_data.generate_one(map_path)
k_model = KeypressModel(None, None, device, k_model_config)
k_predictions = k_model.predict(inputs)

In [8]:
# Post Processing
%autoreload 2
from utils.PostProcess import post_process, save_replay

replay_data = post_process(p_predictions, k_predictions, p_time, k_time, k_end_time)
save_replay(replay_data, '../replay_template.osr', map_path, p_model.name, k_model.name)